#Grupo 9 Práctica 2

#Prerequisitos:

##Import de librerías necesarias

In [34]:
!pip3 install numpy
!pip3 install pandas
!pip3 install scikit-learn
!pip3 install scikit-optimize
!pip3 install skopt
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install statsmodels
!pip3 install lightgbm
!pip3 install xgboost

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import ensemble, tree, linear_model
import time
from scipy.stats import loguniform, randint, uniform
from sklearn import metrics
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Integer
from sklearn.pipeline import Pipeline
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier #CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, balanced_accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
from skopt.space import Integer, Real, Categorical



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement skopt (from versions: none)
ERROR: No matching distribution found for skopt

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


##Extracción de datos

In [21]:
datos = pd.read_pickle('./attrition_available_9.pkl')
target = datos['Attrition']
data = datos.drop(columns=['Attrition'])
#------------------------------------------
#Encodear variables categoricas
#------------------------------------------
data_encoded = pd.get_dummies(data)
ds = data_encoded.T
X = [] 
y = target.T.tolist()
for i in ds.columns:
    X.append(ds[i].tolist())


#EDA

In [ ]:
print(datos.head())
print(datos.info())
print(datos['Attrition'].value_counts(normalize=True))
print(datos.isnull().sum())

datos_no_na = datos.dropna()
datos_no_na.fillna(datos_no_na['cat_var'].mode()[0], inplace=True)
datos_no_na.fillna(datos_no_na['num_var'].median(), inplace=True)

pos_class = datos_no_na[datos_no_na['Attrition'] == 1]
neg_class = datos_no_na[datos_no_na['Attrition'] == 0]

pos_class_upsampled = pos_class.sample(pos_class, n=len(neg_class), replace=True, random_state=42)

datos_balanced = pd.concat([pos_class_upsampled, neg_class])
datos_balanced = datos_balanced.sample(frac=1, random_state=42)

sns.pairplot(datos_balanced, hue='Attrition', palette='Set1')
plt.show()

sns.boxplot(x='Attrition', y='Age', data=datos_balanced)
plt.show()

sns.countplot(x='cat_var', hue='Attrition', data=datos_balanced)
plt.show()

Sabemos que se trata de un problema de clasificación desbalanceado, por lo que se debe tener en cuenta a la hora de evaluar los modelos. Además, se debe tener en cuenta que el objetivo es predecir si un empleado se va a ir de la empresa o no, por lo que se debe tener en cuenta que la métrica a utilizar es la precisión. Missing values, se debe imputar los datos, y se debe utilizar una partición estratificada. 

#Construcción de modelos

##Extracción del conjunto de datos train/test

In [22]:
X_train = X[0:2940]
X_test = X[2940:]
y_train = y[0:2940]
y_test = y[2940:]
X_train_train = X_train[0:1960]
X_train_test = X_train[1960:]
y_train_train = y_train[0:1960]
y_train_test = y_train[1960:]

##Modelo 1: Regresión Logística

In [46]:
#--------------------------------------
# Logistic Regression
#--------------------------------------

inicio = time.time()
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
clf = LogisticRegression(random_state=9, class_weight='balanced')
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', clf)])
pipe_clf.fit(X_train_train, y_train_train)
y_pred = pipe_clf.predict(X_train_test)
f1 = f1_score(y_train_test, y_pred)
bal_acc = balanced_accuracy_score(y_train_test, y_pred)
cm = confusion_matrix(y_train_test, y_pred)
final = time.time()
tiempo = final - inicio
print(f"El tiempo tardado es de {tiempo} segundos.\n")
print(f"El f1 es de {f1}\n")
print(f"El balanced accuracy es de {bal_acc}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Logistic Regression
#--------------------------------------

ValueError: pos_label=1 is not a valid label. It should be one of ['No', 'Yes']

##Modelo 2: Boosting

##Modelo 2.1: Gradient Boosting

In [26]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = {
         'clf__max_depth': [3, 5, 7],
         'clf__learning_rate': [0.1, 0.01, 0.001],
         'clf__subsample': [0.5, 0.8, 1.0],
         'clf__max_features': ['sqrt', 'log2', None],
         'clf__n_estimators': [100, 500, 1000],
         'clf__min_samples_split': [2, 5, 10],
         'clf__min_samples_leaf': [1, 2, 4],
         'clf__min_weight_fraction_leaf': [0.0, 0.1, 0.2]}
imputer = SimpleImputer(strategy='mean')
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = GradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

KeyboardInterrupt: 

In [49]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = {
         'clf__max_depth': randint(low=3, high=10),
         'clf__learning_rate': uniform(0.001, 0.1),
         'clf__subsample': uniform(0.5, 0.5),
         'clf__max_features': ['sqrt', 'log2', None],
         'clf__n_estimators': randint(low=100, high=1000),
         'clf__min_samples_split': randint(low=2, high=20),
         'clf__min_samples_leaf': randint(low=1, high=5),
         'clf__min_weight_fraction_leaf': uniform(0.0, 0.2)}
imputer = SimpleImputer(strategy='mean')
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = GradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

C:\Users\aalex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


El tiempo tardado es de 11.130460739135742 segundos para f1.

El f1 es de nan con estimadores Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', None),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.08124296260447306,
                                            max_depth=9, max_features='sqrt',
                                            min_samples_split=17,
                                            min_weight_fraction_leaf=0.10192161971328487,
                                            n_estimators=299, random_state=9,
                                            subsample=0.7592531456212814))])

El tiempo tardado es de 9.185398578643799 segundos para balanced accuracy.

El balanced accuracy es de 0.8469711385283081 con estimadores Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', None),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.07774173405764452,
                                            

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = {'clf__class_weight': Categorical(categories=['balanced', None], name='clf__class_weight'),
         'clf__max_depth': Integer(low=3, high=10, name='clf__max_depth'),
         'clf__learning_rate': Real(low=0.001, high=0.1, prior='log-uniform', name='clf__learning_rate'),
         'clf__subsample': Real(low=0.5, high=1.0, prior='uniform', name='clf__subsample'),
         'clf__max_features': Categorical(categories=['sqrt', 'log2', None], name='clf__max_features'),
         'clf__n_estimators': Integer(low=100, high=1000, name='clf__n_estimators'),
         'clf__min_samples_split': Integer(low=2, high=20, name='clf__min_samples_split'),
         'clf__min_samples_leaf': Integer(low=1, high=5, name='clf__min_samples_leaf'),
         'clf__min_weight_fraction_leaf': Real(low=0.0, high=0.2, prior='uniform', name='clf__min_weight_fraction_leaf')}
imputer = SimpleImputer(strategy='mean')
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = GradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")


#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

##Modelo 2.2: LightGBM

In [38]:
#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': [50, 100, 500],
              'clf__learning_rate': [0.01, 0.1, 1],
              'clf__max_depth': [3, 5, 7],
              'clf__scale_pos_weight': [1, 10, 25, 50],
              'clf__random_state': [42],
              'clf__class_weight': ['balanced']}
imputer = SimpleImputer(strategy='mean')
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

C:\Users\aalex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


El tiempo tardado es de 11.205840349197388 segundos para f1.

El f1 es de nan con estimadores Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', None),
                ('clf',
                 LGBMClassifier(class_weight='balanced', learning_rate=0.01,
                                max_depth=3, n_estimators=50, random_state=42,
                                scale_pos_weight=1))])

El tiempo tardado es de 9.270069599151611 segundos para balanced accuracy.

El balanced accuracy es de 0.8486382851607356 con estimadores Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', None),
                ('clf',
                 LGBMClassifier(class_weight='balanced', max_depth=7,
                                n_estimators=500, random_state=42,
                                scale_pos_weight=25))])



In [36]:
#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': randint(50, 500),
              'clf__learning_rate': loguniform(0.001, 1),
              'clf__max_depth': randint(3, 7),
              'clf__scale_pos_weight': [1, 10, 25, 50],
              'clf__random_state': [42],
              'clf__class_weight': ['balanced']}
imputer = SimpleImputer(strategy='mean')
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

C:\Users\aalex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


El tiempo tardado es de 1.1591265201568604 segundos para f1.

El f1 es de nan con estimadores Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', None),
                ('clf',
                 LGBMClassifier(class_weight='balanced',
                                learning_rate=0.026241738888884116, max_depth=3,
                                n_estimators=408, random_state=42,
                                scale_pos_weight=10))])

El tiempo tardado es de 1.1525464057922363 segundos para balanced accuracy.

El balanced accuracy es de 0.8444786821075299 con estimadores Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', None),
                ('clf',
                 LGBMClassifier(class_weight='balanced',
                                learning_rate=0.2889166424774813, max_depth=4,
                                n_estimators=455, random_state=42,
                                scale_pos_weight=1))])



In [ ]:
#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': Integer(50, 500),
              'clf__learning_rate': Real(0.001, 1, prior='log-uniform'),
              'clf__max_depth': Integer(3, 7),
              'clf__scale_pos_weight': Categorical([1, 10, 25, 50]),
              'clf__random_state': Constant(42),
              'clf__class_weight': Categorical(['balanced'])}
imputer = SimpleImputer(strategy='mean')
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

##Modelo 2.3: XGBoost

In [ ]:
#--------------------------------------
# Boosting with XGBoost with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': [50, 100, 500],
              'clf__learning_rate': [0.01, 0.1, 1],
              'clf__max_depth': [3, 5, 7],
              'clf__scale_pos_weight': [1, 10, 25, 50],
              'clf__random_state': [42],
              'clf__class_weight': ['balanced']}
imputer = SimpleImputer(strategy='mean')
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = XGBClassifier(random_state=9)
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with XGBoost with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with XGBoost with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': randint(50, 500),
              'clf__learning_rate': loguniform(0.001, 1),
              'clf__max_depth': randint(3, 7),
              'clf__scale_pos_weight': [1, 10, 25, 50],
              'clf__random_state': [42],
              'clf__class_weight': ['balanced']}
imputer = SimpleImputer(strategy='mean')
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = XGBClassifier(random_state=9)
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with XGBoost with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with XGBoost with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': Integer(50, 500),
              'clf__learning_rate': Real(0.001, 1, prior='log-uniform'),
              'clf__max_depth': Integer(3, 7),
              'clf__scale_pos_weight': Categorical([1, 10, 25, 50]),
              'clf__random_state': Constant(42),
              'clf__class_weight': Categorical(['balanced'])}
imputer = SimpleImputer(strategy='mean')
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = XGBClassifier(random_state=9)
pipe_clf = Pipeline([('imputer', imputer), ('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with XGBoost with Hyperparameter Tuning
#--------------------------------------

##Modelo 2.4: CatBoost

In [ ]:
#--------------------------------------
# Boosting with CatBoost with Hyperparameter Tuning
#--------------------------------------

param = {'clf__learning_rate': [0.01, 0.1, 1],
              'clf__n_estimators': [100, 500, 1000],
              'clf__max_depth': [3, 5, 7],
              'clf__class_weights': ['balanced']}
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = CatBoostClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with CatBoost with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with CatBoost with Hyperparameter Tuning
#--------------------------------------

param = {'clf__learning_rate': loguniform(0.001, 1),
              'clf__n_estimators': randint(100, 1000),
              'clf__max_depth': randint(3, 7),
              'clf__class_weights': ['balanced']}
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = CatBoostClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with CatBoost with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with CatBoost with Hyperparameter Tuning
#--------------------------------------

param = {'clf__learning_rate': Real(0.001, 1, prior='log-uniform'),
              'clf__n_estimators': Integer(100, 1000),
              'clf__max_depth': Integer(3, 7),
              'clf__class_weights': Categorical(['balanced'])}
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = CatBoostClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with CatBoost with Hyperparameter Tuning
#--------------------------------------

##Modelo 2.5: AdaBoost

In [ ]:
#--------------------------------------
# Boosting with AdaBoost with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': [50, 100, 500],
              'clf__learning_rate': [0.01, 0.1, 1],
              'clf__algorithm': ['SAMME', 'SAMME.R'],
              'clf__random_state': [42],
              'clf__class_weight': ['balanced']}
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = AdaBoostClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with AdaBoost with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with AdaBoost with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': randint(50, 500),
              'clf__learning_rate': loguniform(0.001, 1),
              'clf__algorithm': ['SAMME', 'SAMME.R'],
              'clf__random_state': [42],
              'clf__class_weight': ['balanced']}
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = AdaBoostClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = RandomizedSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with AdaBoost with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with AdaBoost with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': Integer(50, 500),
              'clf__learning_rate': Real(0.001, 1, prior='log-uniform'),
              'clf__algorithm': Categorical(['SAMME', 'SAMME.R']),
              'clf__random_state': Constant(42),
              'clf__class_weight': Categorical(['balanced'])}
inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scaler = None
b_clf = AdaBoostClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train_train, y_train_train)
final = time.time()
tiempo_bal_acc = final - inicio
# inicio = time.time()
# clf_conf_matrix = BayesSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= '', n_jobs=-1).fit(X_train_train, y_train_train)
# final = time.time()
# tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")

#--------------------------------------
# Boosting with AdaBoost with Hyperparameter Tuning
#--------------------------------------

#Modelo con atributos de tipo filtro

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------



#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------



#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with XGBoost with filter
#--------------------------------------



#--------------------------------------
# Boosting with XGBoost with filter
#--------------------------------------

#Modelos con EDA

##Modelo 1: Regresión Logística

In [ ]:
#--------------------------------------
# Logistic Regression after EDA
#--------------------------------------



#--------------------------------------
# Logistic Regression after EDA
#--------------------------------------

##Modelo 2: Boosting

In [ ]:
#--------------------------------------
# Boosting with (...) after EDA
#--------------------------------------



#--------------------------------------
# Boosting with (...) after EDA
#--------------------------------------

#Conclusiones y comparación de modelos

#Modelo final